# Results

The first effort consisted of:
* Creating a model that expected tripes in the form of (x_hyponym, y_hypernym_positive, y_hypernym_negative)
* The first part of the model learnt features v_i, a relevant hypernym and v_j an irrelevant hypernym
* A projection matrix is trained and the dot product of the projection with the hyponym embedding produces a hyponym feature.
* The vector offset of the correct hypernym and hyponym projection yields a vector corresponding to a relevant document;
* Similarly a vector offset of incorrect hypernym and hyponym projection returns the irrelevant document;
* Subsequently these vectors are inputted into a RankNet model which expects a feature-pair (v_i, v_j) where v_i is considered more relevant than v_j;
* The RankNet model is composed of three fully-connected layers each activated by the rectified linear unit (RELU) non-linear function;
* The relevant and irrelevant vectors pass to each of the three layers to produce a 32-dimensional vector respectively.  The offset of these two vectos is finally piped into a sigmoid-activated layer.  The model minimises log loss and the gradient descent optimizer is provided by Adam

Some basics:
* Dropout set after the hypo, hyper and negative embeddings.  Rate = 0.3;
* Batchsize =32; epochs = 10; adadelta optimizer (?)
* Phi initialised to random identity matrix 

Losses:
1 0.149313178260497 0.14353606931764806
2 0.07674601504530007 0.12045560895597737
3 0.06126764624740355 0.12182305095631454
4 0.052005275971482245 0.09615607964047376
5 0.044895536138555375 0.1055325785721825
6 0.042037001665707216 0.10497323169626725
7 0.040073270672415645 0.09815301130841544
8 0.037451059229814905 0.11646560066671373
9 0.03674421248401006 0.10440275654660945
10 0.03392387550388917 0.08775463453365592

Evaluation:
CRIM evaluation:
MRR: 0.15122
MAP: 0.07179
P@1: 0.11207
P@5: 0.06941
P@10: 0.06568


* let's redo it again but including the validated MRR after each epoch
* adadelta optimizer used.  Better performance - learning rate adapts based "on moving window of gradient updates"

1 0.15895810675456953 0.10530946234986185 0.10286
2 0.09191503602258974 0.08909897886091495 0.069
3 0.07546845667142624 0.09783118630074386 0.09
4 0.06748006921332708 0.0848474074284386 0.128
5 0.06167453078373837 0.0870479880369603 0.144
6 0.06004117455561024 0.10557409031138378 0.09233
7 0.05759255780186686 0.10030873265282025 0.138
8 0.053835369133985775 0.09789049914537463 0.10567
9 0.05120824196036949 0.08685761051625741 0.15556
10 0.04937094883955 0.08225193075121387 0.16556

* let's try for a few more epochs...

1 0.04811658648659698 0.09133973029195112 0.10286
2 0.048253684386725476 0.08190177092889281 0.13119
3 0.04626288837555175 0.09462973917924683 0.09589

Performance degrades again...i think i should run for full stretch of epochs and not stop and restart;
Might upset internal state of optimizer


* Here I'm redoing it again but I'm including MRR and MAP after every epoch.  Also, tweaked the evaluation mechanism to consider all evaluations.  Need to be able to capture best model and save it so I can easily apply the model and evaluation on test.

1 0.1577925027145454 0.09330397779028862 0.09127 0.05612
2 0.0902454890253204 0.07547199155064299 0.09252 0.07953
3 0.0757413568341557 0.07951848581479862 0.11632 0.07015
4 0.06698930700853077 0.09419791811874893 0.13983 0.09942
5 0.06188068899102822 0.10137247232125082 0.11539 0.07119
6 0.05901100889947485 0.07420569466988672 0.11325 0.07933
7 0.0554251948882061 0.09873672267608345 0.11091 0.07439
8 0.05224523330387226 0.0838388080745499 0.14841 0.08781
9 0.05268218255786957 0.0884869313114832 0.18317 0.10689
10 0.049139027892415754 0.08971231535818515 0.11636 0.07904


* we try to normalise the phi matrix prior to performing the vector offset with the hypernym

Losses:
1
2
3
4
5
6 0.05055866682434617 0.09070048343630333 0.07
7 0.044227731492333916 0.07728665156071383 0.16619
8 0.04071681306900273 0.04910805218011956 0.13533
9 0.037880599411295464 0.07110782440233743 0.12667
10 0.03477762145808523 0.07052839489196686 0.137

* fares somewhat better with the adadelta optimizer; although beyond a certain point starts to degrade

1 0.16768017700874185 0.11283031322516035 0.05333
2 0.09299927773789837 0.10060590638438589 0.1
3 0.074038846712673 0.10403750901406601 0.09094
4 0.06307647197995715 0.11161908269934065 0.135
5 0.05512060815501164 0.0796422848692273 0.06186
6 0.0502728624920873 0.09669758699843806 0.13667
7 0.045994018233337805 0.08753541226370067 0.105
8 0.04198638971548115 0.07141121624017978 0.13333
9 0.038961860982876156 0.08742369918344821 0.17167
10 0.03710430475306753 0.07209413202800898 0.126
11 0.03403899900376672 0.07026687171170716 0.13833
12 0.030937558923702018 0.07371389029574767 0.1375
13 0.029745133011300426 0.06735533633845625 0.10619


# Added dropout after first two RankNet Dense layers

* Same parameters; adadelta optimizer (Default); Dropout rate is 0.3 throughout;

* Adding dropout seems to have the effect of stabilising the output.  
* Loss slows down after immediately scoring a relativel high result after the first epoch.
* After the first epoch result starts to degrade consistently then picks up slightly towards the middle of training;
* Beyond first 10 epochs, MRR starts to degrade consistently

1 0.21446231449031902 0.12595248990738764 0.14983 0.07389
2 0.14248283636879663 0.11914396967831999 0.14833 0.0731
3 0.12308801912844702 0.11549847886525094 0.12686 0.06004
4 0.1146570320401737 0.09902715774881654 0.115 0.0558
5 0.11031864524312696 0.08954297142545693 0.17533 0.07701
6 0.10536578997054988 0.10434427518458687 0.14033 0.06489
7 0.10207657396764269 0.08509728500503115 0.14333 0.08151
8 0.09930220836415661 0.10186002227454447 0.17 0.08072
9 0.09512085899016769 0.09900960580568062 0.16067 0.06702
10 0.09262682785359827 0.07961805544936215 0.20619 0.10465
11 0.09410519668401826 0.08357024383140378 0.16067 0.08743
12 0.09225273876879417 0.08443483309185831 0.12 0.05228
13 0.09154559603710967 0.08245583580661332 0.11833 0.06015

* Multiple runs yields different results
1 0.22004186438627604 0.12888010836904867 0.09067 0.04443
2 0.14342507004082553 0.09926815450983122 0.05333 0.02778
3 0.1252973076674345 0.09981001526699401 0.07333 0.03794
4 0.11623276932371707 0.090312896126532 0.11667 0.05939
5 0.10819901013298551 0.08604736268892885 0.11833 0.05756
6 0.10451369480554798 0.08490674913616385 0.12286 0.05381
7 0.10084040144674897 0.08725397952905041 0.11619 0.05708
8 0.0990454405765044 0.09215930400614161 0.11 0.05625
9 0.09660953868568571 0.07978438950981945 0.11333 0.05229
10 0.09247438810143965 0.09442004908632953 0.11 0.0512

* Inspection of the actual predictions are disappointing


# Squaring the vector offset

* adam optimizer with same parameters CRIM use (betas set to 0.9; clipnorm set to 1.)
* no phi normalisation before squaring the vector offset

1 0.18278130501469494 0.12448017159849405 0.105
2 0.11185156123091039 0.1145224037207663 0.1225
3 0.09746940114245771 0.11936045041959732 0.0875
4 0.0899683687726504 0.11530433653248474 0.10783
5 0.08775127774729258 0.1291229133279994 0.11067
6 0.0853185890610116 0.12455064459796995 0.07148
7 0.08134474724281957 0.11269128893548623 0.07019
8 0.08052958931607229 0.11925153115205467 0.08852
9 0.07895006665553395 0.1228176112731453 0.10369
10 0.07689826459233998 0.1202320301909931 0.07144

* same as above but with adadelta optimizer

1 0.18422494277045584 0.1352659973828122 0.107 0.05616
2 0.11100932204712731 0.13118468849733472 0.05667 0.0273
3 0.09646434022216625 0.1225130601497367 0.06286 0.02743
4 0.08799796130481935 0.11970055847521871 0.08133 0.03615
5 0.08467672832249477 0.1155472748875618 0.1 0.03336
6 0.08202950266642212 0.1224163043834269 0.07067 0.02792
7 0.07940323380724143 0.11635909711569548 0.083 0.03507
8 0.07741339176811637 0.11734444469073788 0.074 0.03557
9 0.07568845656381083 0.12491223653499037 0.07717 0.03781
10 0.07491118155801875 0.12184002135321498 0.07556 0.04189


* after modifying evaluation script to eliminate bias on terms having very few gold-standard hypernyms
1 0.18715833974571258 0.14324840383417906 0.17048 0.09256
2 0.10985544529487397 0.12990571602014825 0.12294 0.06928
3 0.09624372861367939 0.12559192659612745 0.11885 0.07235
4 0.08767683944836549 0.12156628941791132 0.12167 0.06751
5 0.08332296796876791 0.13745410638395697 0.14021 0.07853
6 0.0806314828946917 0.1273054619487375 0.08722 0.05323
7 0.0775965833403148 0.1240518649192527 0.08487 0.06371
8 0.07652003470928764 0.12674982292344794 0.11205 0.07673
9 0.07616731932158886 0.12484497489826754 0.1102 0.09637
10 0.07443902538414886 0.12584021718334407 0.14364 0.11692



* adam optimizer with same parameters CRIM use (betas set to 0.9; clipnorm set to 1.)
* normalisation of phi + squaring of the offset vectors
* element-wise squaring the subtraction vectors sets us off to a good start but performance dwindles soon after:

1 0.19140405790578763 0.15374121964909138 0.085
2 0.134404292674717 0.1289560331553221 0.12333
3 0.120723139242847 0.14781217233464122 0.08333
4 0.11038369191214724 0.13470988153852523 0.09267
5 0.10611982218069166 0.13086557064950466 0.08167
6 0.10020121730002542 0.12256309086084366 0.09333
7 0.09876267333337958 0.12047445200942457 0.08
8 0.0957780601457552 0.11608536665327847 0.10667
9 0.09178295032091907 0.13583790874108673 0.059
10 0.08783703389520561 0.1527293598651886 0.075


# No Dropout

* adam optimizer; dropout after phi removed

1 0.16074451444301635 0.13418584796087815 0.044
2 0.09165465517015578 0.11261159054466406 0.12
3 0.07516317882114375 0.12670742241537664 0.06333
4 0.0687957276668365 0.12111110112303868 0.0375
5 0.06378151471821081 0.13718547264439984 0.065
6 0.05917153576522103 0.11989653448415265 0.06367
7 0.056860829926714124 0.1300455170768928 0.05
8 0.055901598586641635 0.13728181337341083 0.02067
9 0.054057386066846506 0.1316339255715684 0.08
10 0.05239380398415417 0.13197182397090002 0.05784


# multi-phi
* In first experiment, I increased the number of projection matrices
* Each projection matrix yielded a projected hyponym
* I took the mean of each projection hyponym, intending it to be the average projection from all matrices
* The feature vector is the offset between correct hypernym and mean projected hyponym
* The offset vector was squared, element-wise
* Increased the number of negatives to 15

1 0.1638881492075503 0.12348213917668908 0.07702 0.04856
2 0.07609728164220271 0.10782999052200466 0.08515 0.05931
3 0.05838024439703156 0.09932488317104678 0.07693 0.05396
4 0.05073596857114751 0.10930847434447302 0.07783 0.0564
5 0.04623325555859553 0.11539426855246226 0.10919 0.07058
6 0.04314579680827366 0.11579047930023323 0.07522 0.05484
7 0.03974381190856632 0.12110452579439152 0.09756 0.07884
8 0.03874994410668346 0.10645054051481809 0.09656 0.07225
9 0.036352846745428144 0.11111807118284438 0.10119 0.07283
10 0.03615959057760908 0.11030580726234863 0.12802 0.08279

CRIM evaluation:
MRR: 0.1551
MAP: 0.09707
P@1: 0.0974
P@5: 0.08143
P@10: 0.10474

* reduced the negatives to 10
* added dropout to the ranknet model (0.2 rate)

1 0.22124181946929156 0.1416767184752971 0.12536 0.08431
2 0.11066557578536931 0.12897819087002427 0.11153 0.07879
3 0.08757654473597316 0.1342262463759398 0.11722 0.09453
4 0.07511341618589841 0.13301655492931605 0.13269 0.09368
5 0.06757064723476207 0.1297981228029239 0.10169 0.08451
6 0.06435743415841592 0.1245770838993485 0.09784 0.07283
7 0.0612306901270601 0.121899301397847 0.09434 0.07366
8 0.0579709924974755 0.12992136604548432 0.15123 0.08615
9 0.05450322283058694 0.12041602811828489 0.09863 0.0659
10 0.053598353442391555 0.12179123197554145 0.10998 0.08446

CRIM evaluation:
MRR: 0.18502
MAP: 0.10741
P@1: 0.13009
P@5: 0.08978
P@10: 0.10976




# Tried fine-tuning embeddings alongside projection matrix and ranker

Train on 117790 samples, validate on 2000 samples
Epoch 1/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.1961; MRR: 0.1187; MAP: 0.0809
117790/117790 [==============================] - 3599s 31ms/step - loss: 0.1961 - val_loss: 0.1234
Epoch 2/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0634; MRR: 0.1166; MAP: 0.0769
117790/117790 [==============================] - 3588s 30ms/step - loss: 0.0634 - val_loss: 0.1391
Epoch 3/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0405; MRR: 0.1231; MAP: 0.0753
117790/117790 [==============================] - 3572s 30ms/step - loss: 0.0405 - val_loss: 0.1317
Epoch 4/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0276; MRR: 0.1383; MAP: 0.0761
117790/117790 [==============================] - 3546s 30ms/step - loss: 0.0276 - val_loss: 0.1372
Epoch 5/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0220; MRR: 0.1501; MAP: 0.0810
117790/117790 [==============================] - 3601s 31ms/step - loss: 0.0220 - val_loss: 0.1221
Epoch 6/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0183; MRR: 0.1763; MAP: 0.0928
117790/117790 [==============================] - 3581s 30ms/step - loss: 0.0183 - val_loss: 0.1258
Epoch 7/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0153; MRR: 0.1331; MAP: 0.0750
117790/117790 [==============================] - 3559s 30ms/step - loss: 0.0153 - val_loss: 0.1160
Epoch 8/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0132; MRR: 0.1079; MAP: 0.0729
117790/117790 [==============================] - 3566s 30ms/step - loss: 0.0132 - val_loss: 0.1208
Epoch 9/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0115; MRR: 0.1313; MAP: 0.0811
117790/117790 [==============================] - 3611s 31ms/step - loss: 0.0115 - val_loss: 0.1281
Epoch 10/10
117760/117790 [============================>.] - ETA: 0s - loss: 0.0103; MRR: 0.1340; MAP: 0.0770
117790/117790 [==============================] - 3548s 30ms/step - loss: 0.0103 - val_loss: 0.1210


CRIM evaluation:
MRR: 0.172
MAP: 0.10266
P@1: 0.11875
P@5: 0.08724
P@10: 0.10728

